https://beckernick.github.io/matrix-factorization-recommender/

In [1]:
#importing relevant packages and modules
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#setting default plot settings
sns.set(rc={'figure.figsize':(10,7)}, font_scale=1.3)

In [3]:
#accessing the local directory for the data
PATH = os.path.join(os.environ['HOMEPATH'], 'data', 'amazon_cleaned.csv')
df = pd.read_csv(PATH, index_col=0)

C:\Users\Thompson\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
df.head()

,itemID,rating,reviewText,reviewTime,reviewerID,summary,foundHelpful,totalHelpful
0,0528881469,5,We got this GPS for my husband who is an (OTR)...,2013-06-02,AO94DHGC771SJ,Gotta have GPS!,0,0
1,0528881469,1,"I'm a professional OTR truck driver, and I bou...",2010-11-25,AMO214LNFCEI4,Very Disappointed,12,15
2,0528881469,3,"Well, what can I say. I've had this unit in m...",2010-09-09,A3N7T0DY83Y4IG,1st impression,43,45
3,0528881469,2,"Not going to write a long review, even thought...",2010-11-24,A1H8PY3QHMQQA0,"Great grafics, POOR GPS",9,10
4,0528881469,1,I've had mine for a year and here's what we go...,2011-09-29,A24EV6RXELQZ63,"Major issues, only excuses for support",0,0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1689188 entries, 0 to 1689187
Data columns (total 8 columns):
itemID          1689188 non-null object
rating          1689188 non-null int64
reviewText      1688117 non-null object
reviewTime      1689188 non-null object
reviewerID      1689188 non-null object
summary         1689173 non-null object
foundHelpful    1689188 non-null int64
totalHelpful    1689188 non-null int64
dtypes: int64(3), object(5)
memory usage: 116.0+ MB


In [10]:
relevant_columns = df[['itemID', 'rating', 'reviewerID']]
relevant_columns.head()

,itemID,rating,reviewerID
0,0528881469,5,AO94DHGC771SJ
1,0528881469,1,AMO214LNFCEI4
2,0528881469,3,A3N7T0DY83Y4IG
3,0528881469,2,A1H8PY3QHMQQA0
4,0528881469,1,A24EV6RXELQZ63


In [15]:
relevant_columns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1689188 entries, 0 to 1689187
Data columns (total 3 columns):
itemID        1689188 non-null object
rating        1689188 non-null int64
reviewerID    1689188 non-null object
dtypes: int64(1), object(2)
memory usage: 51.5+ MB


In [16]:
test_pivot = relevant_columns.pivot(columns='itemID', index='reviewerID', values='rating')
test_pivot.info()

ValueError: Unstacked DataFrame is too big, causing int32 overflow

pyspark notebooks, python notebooks using spark
filter dataset to lower years

In [ ]:
.reshape(-1,1), numpy

In [5]:
#item = df[['itemID']].astype('category')
#reviewer = df[['reviewerID']].astype('category')
#rating = df[['rating']].astype('category')

In [6]:
#https://stackoverflow.com/questions/31661604/efficiently-create-sparse-pivot-tables-in-pandas
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

reviewer = CategoricalDtype(sorted(df.reviewerID.unique()), ordered=True)
item = CategoricalDtype(sorted(df.itemID.unique()), ordered=True)

#pandas.astype('categorical')
#index for ID

row = df.reviewerID.astype(reviewer).cat.codes
col = df.itemID.astype(item).cat.codes
sparse_matrix = csr_matrix((df['rating'],(row,col)), shape=(reviewer.categories.size, item.categories.size))


In [7]:
sparse_matrix

<192403x63001 sparse matrix of type '<class 'numpy.int64'>'
	with 1689188 stored elements in Compressed Sparse Row format>

In [8]:
df_sparse = pd.SparseDataFrame(sparse_matrix, index=reviewer.categories, columns=item.categories, default_fill_value=0)

C:\Users\Thompson\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  """Entry point for launching an IPython kernel.
C:\Users\Thompson\Anaconda3\lib\site-packages\pandas\core\sparse\frame.py:257: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  sparse_index=BlockIndex(N, blocs, blens),


In [9]:
df_sparse.head()

C:\Users\Thompson\Anaconda3\lib\site-packages\pandas\core\frame.py:3471: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  return klass(values, index=self.index, name=items, fastpath=True)
C:\Users\Thompson\Anaconda3\lib\site-packages\pandas\core\sparse\frame.py:745: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  default_fill_value=self._default_fill_value,
C:\Users\Thompson\Anaconda3\lib\site-packages\pandas\core\sparse\frame.py:721: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays inst

,0528881469,0594451647,0594481813,0972683275,1400501466,1400501520,1400501776,1400532620,1400532655,140053271X,...,B00KSLCU72,B00KVNY2KA,B00KWHMR6G,B00KXAFYZS,B00KYMCJF8,B00L21HC7A,B00L2442H0,B00L26YDA4,B00L3YHF6O,B00LGQ6HL8
A000715434M800HLCENK9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00101847G3FJTWYGNQA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00166281YWM98A3SVD55,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0046696382DWIPVIWO0K,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00472881KT6WR48K907X,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
df_sparse.info()

<class 'pandas.core.sparse.frame.SparseDataFrame'>
Index: 192403 entries, A000715434M800HLCENK9 to AZZYW4YOE1B6E
Columns: 63001 entries, 0528881469 to B00LGQ6HL8
dtypes: Sparse[int64, 0](63001)
memory usage: 32.2+ MB


In [36]:
relevant_columns = df[['itemID', 'rating', 'reviewerID']]
relevant_columns.head()

In [6]:
relevant_columns.head()

,itemID,rating,reviewerID
0,0528881469,5,AO94DHGC771SJ
1,0528881469,1,AMO214LNFCEI4
2,0528881469,3,A3N7T0DY83Y4IG
3,0528881469,2,A1H8PY3QHMQQA0
4,0528881469,1,A24EV6RXELQZ63


In [17]:
relevant_columns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1689188 entries, 0 to 1689187
Data columns (total 3 columns):
itemID        1689188 non-null object
rating        1689188 non-null int64
reviewerID    1689188 non-null object
dtypes: int64(1), object(2)
memory usage: 51.5+ MB


In [40]:
relevant_columns.loc[:,'itemID'] = relevant_columns['itemID'].astype('category')
relevant_columns.loc[:,'reviewerID'] = relevant_columns['reviewerID'].astype('category')
cat_columns = relevant_columns

In [41]:
cat_columns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1689188 entries, 0 to 1689187
Data columns (total 3 columns):
itemID        1689188 non-null category
rating        1689188 non-null int64
reviewerID    1689188 non-null category
dtypes: category(2), int64(1)
memory usage: 48.1 MB


In [42]:
cat_col = cat_columns.head(300000)
cat_col_pivot = cat_col.pivot(columns='itemID', index='reviewerID', values='rating')

In [43]:
cat_col_pivot.info()

<class 'pandas.core.frame.DataFrame'>
CategoricalIndex: 123257 entries, AO94DHGC771SJ to A3M78605OXJY8P
Columns: 12401 entries, 0528881469 to B000R2PC2M
dtypes: float64(12401)
memory usage: 11.4 GB


In [11]:
test = relevant_columns.head(300000)
test_pivot = test.pivot(columns='itemID', index='reviewerID', values='rating')
test_pivot.info()

In [8]:
test_pivot = test.pivot(columns='itemID', index='reviewerID', values='rating')
test_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Index: 97809 entries, A00472881KT6WR48K907X to AZZYW4YOE1B6E
Columns: 8381 entries, 0528881469 to B000EUGX70
dtypes: float64(8381)
memory usage: 6.1+ GB


In [9]:
test_pivot.to_sparse().info()

C:\Users\Thompson\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: DataFrame.to_sparse is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
C:\Users\Thompson\Anaconda3\lib\site-packages\pandas\core\frame.py:3471: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  return klass(values, index=self.index, name=items, fastpath=True)


<class 'pandas.core.sparse.frame.SparseDataFrame'>
Index: 97809 entries, A00472881KT6WR48K907X to AZZYW4YOE1B6E
Columns: 8381 entries, 0528881469 to B000EUGX70
dtypes: Sparse[float64, nan](8381)
memory usage: 3.8+ MB


In [12]:
pd.pivot_table(test, columns='itemID', index='reviewerID', values='rating', observed=True)

MemoryError: 

In [ ]:
sdf = df.astype(pd.SparseDtype("float", np.nan))

In [28]:
#relevant_columns.pivot(index='reviewerID', columns='itemID', values='rating')
pivot_df = pd.pivot_table(relevant_columns, values='rating', index='reviewerID', columns='itemID').to_sparse()

#unstacked dataframe is too big, need to some how decrease size

ValueError: Unstacked DataFrame is too big, causing int32 overflow

In [14]:
relevant_columns.to_matrix()

AttributeError: 'DataFrame' object has no attribute 'to_matrix'